In [112]:
from google.colab import drive
drive.mount('/content/drive')
%run "/content/drive/MyDrive/Research/Masters Thesis/V1/0. Parent Code.ipynb"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2401


# 1. Loading Packages

In [113]:
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from statistics import mean

import scipy
from scipy.sparse import csr_matrix

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 2. Select the required texts from LinkedIn

In [114]:
#Create labeled and unlabeled text list 


# sex, name, degree_connection, current_location, current_industry, number_connection, 
# edu1, edu2, edu3, edu4, edu5, edu6, 
# number_skills, skills_list, skills_endorsement_pair, Intersts, summary,
# employment

# Education is iteself and object with the following attributes
# SName, SLevel, SDates, SField, SDegree

#Here employment is a list of jobs where each job contains information below 
# [title, company_name, job_dates, job_location, job_summary, job_skills]


labeled_list = []
labeled_list_label = []
unlabeled_list = []
object_number =0
info_list_labeled = []
info_list_unlabeled = []
for ind in object_list:

    for job in ind.employment:
        if job[4]!=None and job[5]!=None and len(job[5]) >= 5:
            split_description = job[4].split("\n skill : ")
            if len(split_description) == 2 :
                info_list_labeled.append([ object_number ]+ job + [ind.skills_endorsement_pair])
                labeled_list.append(job[4])
                labeled_list_label.append(job[5])
            else:
                print(split_description)
                a = "Error Check" + 1
            
        elif  job[4]!=None: 
            info_list_unlabeled.append([ object_number ]+ job)
            unlabeled_list.append(job[4])
    
    object_number = object_number  +1

    # print("______________________")

print(len(labeled_list))
print(len(info_list_labeled))
print(len(unlabeled_list))
print(len(info_list_unlabeled))

1171
1171
14182
14182


In [115]:
info_list_unlabeled[318]

[60,
 'Study Planning and Feasibility Specialist',
 ['The Steely Group', None],
 ['Feb 2017 – Aug 2017', '6 mos'],
 'King Of Prussia, Pennsylvania',
 '\n • support process for initiate study planning and feasibility activity , include internal and industry search and work with manager to compile datum drive study assumption from protocol go through feasibility completion \n • support of the feasibility ( study , country and site ) process \n • maintain continuity across study by distribute survey and compile feedback on result \n • involve relevant stakeholder in the feasibility process to ensure the all need be consider ensure effective scenario generation & planning ; compare CSLB ’s past performance datum and propose scenario from Industry or vendor datum \n • drive accurate Enrollment Planning . \n • support Clinical Operations early in the development cycle , provide initial enrollment timeline , potential Country Footprint and realistic study enrollment plan be develop and in pla

In [116]:
info_list_labeled[5]

[21,
 'Assistant Director Of Admissions',
 ['The Art Institutes', None],
 ['Nov 2016 – Jul 2018', '1 yr 8 mos'],
 'The Art Instititute of Philadelphia',
 '\n • manage inquiry to achieve prompt contact \n • through solid time management skill , schedule and conduct interview to identify and pursue qualified student \n • consistently conduct follow - up with all applicant to ensure successful matriculation \n • participate in recruitment and enrollment activity include : open house , training session , orientation program , etc . \n\n\n skill : Consultative sale • employee training • Team Building • cold calling • Customer Relationship Management ( CRM ) • customer satisfaction • onboarding ',
 ['Consultative Sales',
  'Employee Training',
  'Team Building',
  'Cold Calling',
  'Customer Relationship Management (CRM)',
  'Customer Satisfaction',
  'Onboarding'],
 [('HubSpot', 0),
  ('Educational Technology', 0),
  ('Consultative Sales', 0),
  ('Start-up Environment', 0),
  ('Account Mana

# 3. Creating three skills from O*NET

## 3.1 Preping : Loading Files, cleaning columns and reordering them

In [117]:
ONET_df = pd.read_csv(main_link  + "Input/final_ONET.csv")
ONET_column_type = ["Abilities", "Work Context", "Skills", "Knowledge", "Work Activities","Description", 	"Rating"]
selected_ONET_column = [x for x in ONET_df.columns if x.split("_")[0] in ONET_column_type ]

ONET_df = ONET_df[selected_ONET_column]

In [118]:
ONET_column_type_skills = ["Abilities", "Work Context", "Skills", "Knowledge", "Work Activities"]
ONET_skills_column = [x for x in ONET_df.columns if x.split("_")[0] in ONET_column_type_skills]


In [119]:
ONET_skills_column.remove('Knowledge_Mathematics')
ONET_skills_column.remove('Skills_Social Perceptiveness')
ONET_skills_column.remove('Knowledge_Mechanical')
modified_ONET_skills_column = ['Knowledge_Mathematics','Knowledge_Mechanical','Skills_Social Perceptiveness' ]+ ONET_skills_column



## 3.2 Creating the modified factors, saving the loadings and PCA eigenvectors into respective variables 

In [120]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca.fit(ONET_df[modified_ONET_skills_column])

pca.explained_variance_ratio_
# #Eigevalues
# print(pca.explained_variance_)
# #Eigemvectors 
# print(len(pca.components_))
# print(len(pca.components_[0]))
# #Factor Loadings
# print(len( pca.components_.T * np.sqrt(pca.explained_variance_)) )
# print(len( pca.components_.T * np.sqrt(pca.explained_variance_)[0]) )
# #Principal Component 
# print(len(pca.fit_transform(ONET_df[modified_ONET_skills_column])))
# print(len(pca.fit_transform(ONET_df[modified_ONET_skills_column])[0]))

# print(pca.fit_transform(ONET_df[modified_ONET_skills_column]))
# print(pca.components_.T * np.sqrt(pca.explained_variance_)) 



array([0.37945627, 0.14309602, 0.07565001])

In [121]:
#Factor Loadings
F3 = np.asarray( pca.fit_transform(ONET_df[modified_ONET_skills_column]))


L3 = np.asarray(pca.components_.T * np.sqrt(pca.explained_variance_)).transpose()

T = L3[:, 0:3]

In [ ]:
from numpy.linalg import inv

print(len(F3))
print(len(F3[0]))
print(len(T))
print(len(T[0]))
print(len(L3))
print(len(L3[0]))

M= np.asarray(ONET_df[modified_ONET_skills_column])
F3_T = np.matmul(F3  , T)
invT_L3 = np.matmul(inv(T)  , L3)
print(F3_T.shape)
print(M.shape)
print(invT_L3.shape)
ONET_3skills_PCA =  F3_T

## 3.3 Final cleaning and saving 

In [76]:
ONET_3skills = []

Y = ONET_df["Description"].values.tolist()
Z = ONET_3skills_PCA.tolist()

for i in range(0,len(Z)):
  ONET_3skills.append( [Y[i]]    + Z[i])

ONET_3skills_df = pd.DataFrame(ONET_3skills, columns = ["text","PC1","PC2","PC3"])


In [77]:
final_ONET = pd.read_csv(main_link + 'Input/final_ONET.csv')
final_ONET["Title"]
ONET_result = pd.concat([final_ONET["Title"], ONET_3skills_df ], axis=1)


ONET_result = ONET_result.rename(columns={"PC1": "ONET_Cognitive", "PC2": "ONET_Mechanical","PC3":"ONET_Interpersonal"})

In [78]:
with pd.ExcelWriter(main_link + 'Temp/ONET_3skills_wIdentifier.xlsx') as writer:  
    ONET_result.to_excel(writer, sheet_name='Sheet_name_1')

# 4. Creating two skills from LinkedIn 

## 4.1 Preping: Loading files, reordering columns

In [79]:
Y_df = pd.read_excel(main_link + 'Input/Y_df.xlsx')
Y_df= Y_df.drop(['Unnamed: 0'], axis=1)

In [80]:
LinkedIn_skills_col= Y_df.columns.tolist()
LinkedIn_skills_col.remove("LinkedIn_Text")

LinkedIn_skills_col.remove('Abilities_Mathematical Reasoning')
LinkedIn_skills_col.remove("Skills_Social Perceptiveness")
modified_LinkedIn_skills_col = ['Abilities_Mathematical Reasoning' , "Skills_Social Perceptiveness"]+ LinkedIn_skills_col

M_LinkedIn= np.asarray(Y_df[modified_LinkedIn_skills_col])

In [81]:
# ['Knowledge_Mathematics','Skills_Social Perceptiveness' ]+ ONET_skills_column

## 4.2 Creating the modified factors, saving the loadings and PCA eigenvectors into respective variables 

In [82]:
pca = PCA(n_components=2)
pca.fit(Y_df[LinkedIn_skills_col])

F2_LinkedIn = np.asarray( pca.fit_transform(Y_df[LinkedIn_skills_col]))
L2_LinkedIn = np.asarray(pca.components_.T * np.sqrt(pca.explained_variance_)).transpose()
T_LinkedIn = L2_LinkedIn[:, 0:2]

pca.explained_variance_ratio_

array([0.51502313, 0.15098155])

In [109]:

PC_to_consider = L2_LinkedIn[0].tolist()
copy_PC_to_consider = PC_to_consider.copy()
copy_PC_to_consider.sort(reverse=True)
highest_factor_loadings = copy_PC_to_consider[:15]


for i in highest_factor_loadings:
  index_of_loading = PC_to_consider.index(i)
  print(i, LinkedIn_skills_col[index_of_loading])

0.22904237771158525 Skills_Speaking
0.22064220929881184 Abilities_Speech Clarity
0.21567614116014752 Abilities_Oral Expression
0.20719969004361194 Skills_Active Listening
0.20719504060775681 Abilities_Speech Recognition
0.19955590939556053 Abilities_Oral Comprehension
0.06663217048339738 Abilities_Inductive Reasoning
0.06059960797851372 Skills_Complex Problem Solving
0.03979081250350892 Abilities_Number Facility


In [84]:
pca.fit_transform(Y_df[LinkedIn_skills_col])

array([[ 0.47846209, -0.34842341],
       [-0.72386611, -0.14011057],
       [ 0.05583748, -0.27903861],
       ...,
       [-1.25348778,  0.55926993],
       [ 0.24638722,  0.3250437 ],
       [ 0.60824801,  0.43921924]])

In [85]:
from numpy.linalg import inv

F2_T_LinkedIn = np.matmul(F2_LinkedIn  , T_LinkedIn)
invT_L2_LinkedIn = np.matmul(inv(T_LinkedIn)  , L2_LinkedIn)
print(F2_T_LinkedIn.shape)
print(M_LinkedIn.shape)
print(invT_L2_LinkedIn.shape)
LinkedIn_2skills_PCA =  F2_T_LinkedIn

(15353, 2)
(15353, 11)
(2, 9)


In [86]:
LinkedIn_2skills = []

Y = Y_df["LinkedIn_Text"].values.tolist()
Z = LinkedIn_2skills_PCA.tolist()

for i in range(0,len(Z)):
  LinkedIn_2skills.append( [Y[i]]    + Z[i])


LinkedIn_2skills_df = pd.DataFrame(LinkedIn_2skills, columns = ["text","PC1","PC2"])

In [87]:
LinkedIn_2skills_df

,text,PC1,PC2
0,\n ● Ran the outbound manual sort and notice t...,-0.040760,-0.041465
1,\n -complete an intensive 6 month technical pr...,-0.077444,-0.053133
2,\n • greeter / ticket taker at sporting event ...,-0.054454,-0.046233
3,"\n ● lead small team on register , where I nee...",-0.040760,-0.041465
4,"\n Casetext , the most widely use 1.I. legal r...",-0.054552,-0.049894
...,...,...,...
15348,\n as 1 business consultant with extensive hea...,0.116856,0.093820
15349,"\n lead the Competitive Intelligence team , pr...",0.116856,0.093820
15350,\n provide thought leadership as 1 senior memb...,0.033076,0.047240
15351,\n support internal customer during all phase ...,0.084184,0.066248


In [88]:
with pd.ExcelWriter(main_link + 'Temp/LinkedIn_2skills_df.xlsx') as writer:  
    LinkedIn_2skills_df.to_excel(writer, sheet_name='Sheet_name_1')

In [89]:
values = LinkedIn_2skills_df["PC1"].values.tolist()
ind = [x for x in range(0,len(values))]
print(pd.Series(values, index = ind).describe())

values = LinkedIn_2skills_df["PC2"].values.tolist()
ind = [x for x in range(0,len(values))]
print(pd.Series(values, index = ind).describe())

count    1.535300e+04
mean     2.082617e-18
std      6.973642e-02
min     -1.843530e-01
25%     -5.706048e-02
50%     -9.497564e-04
75%      5.454866e-02
max      3.882375e-01
dtype: float64
count    1.535300e+04
mean     1.851215e-18
std      5.431535e-02
min     -1.372020e-01
25%     -4.937882e-02
50%      2.681296e-03
75%      3.907928e-02
max      3.011406e-01
dtype: float64


## 4.3 Adding identifiers and saving

In [90]:
# [106,
#  'Application Support Specialist',
#  ['JPMorgan Chase & Co.', None],
#  ['Nov 2015 – Feb 2017', '1 yr 3 mos'],
#  'Wilmington, Delaware, United States',
#  '\n • coordinated deployment and software release , produce change request and conduct comprehensive impact analysis . \n • responsible as the 1st point of contact to review defect for different project ; assign weakness to appoint schema job family , manage the fault SLAs and provide daily update on the fault to management and team . \n • collaborate with other group and organization to provide service for infrastructure change , and improve and sustain the datum quality of the application . ',
#  None]

In [91]:
title_text_pair = []
for job in info_list_unlabeled + info_list_labeled:
  title_text_pair.append((job[5], [job[1],job[3]]))

Dict_text_title = dict(title_text_pair)

In [92]:
LinkedIn_2skills_df["Title"] =  LinkedIn_2skills_df["text"].apply(lambda x: Dict_text_title[x][0])
LinkedIn_2skills_df["Tenure"] =  LinkedIn_2skills_df["text"].apply(lambda x: Dict_text_title[x][1])

In [93]:
LinkedIn_2skills_df

,text,PC1,PC2,Title,Tenure
0,\n ● Ran the outbound manual sort and notice t...,-0.040760,-0.041465,Process Assistant,"[Sep 2015 – Present, 6 yrs 11 mos]"
1,\n -complete an intensive 6 month technical pr...,-0.077444,-0.053133,Frontend Web Developer (Student),"[Jul 2020 – Feb 2021, 7 mos]"
2,\n • greeter / ticket taker at sporting event ...,-0.054454,-0.046233,Greeter/Ticket Taker,"[Feb 2008 – Nov 2015, 7 yrs 9 mos]"
3,"\n ● lead small team on register , where I nee...",-0.040760,-0.041465,Cash Rep,"[Dec 2012 – Nov 2014, 1 yr 11 mos]"
4,"\n Casetext , the most widely use 1.I. legal r...",-0.054552,-0.049894,Account Executive,"[Feb 2022 – Present, 6 mos]"
...,...,...,...,...,...
15348,\n as 1 business consultant with extensive hea...,0.116856,0.093820,Principal Consultant | Competitive Intelligenc...,"[2020 – Present, 2 yrs 6 mos]"
15349,"\n lead the Competitive Intelligence team , pr...",0.116856,0.093820,"Competitive Intelligence, Sr. Manager","[2015 – 2020, 5 yrs]"
15350,\n provide thought leadership as 1 senior memb...,0.033076,0.047240,"Competitive Intelligence, Market Research and ...","[2004 – 2015, 11 yrs]"
15351,\n support internal customer during all phase ...,0.084184,0.066248,Contract Administrator,"[May 2016 – Present, 6 yrs 2 mos]"


In [94]:

with pd.ExcelWriter(main_link + 'Temp/LinkedIn_2skills_df.xlsx') as writer:  
    LinkedIn_2skills_df.to_excel(writer, sheet_name='Sheet_name_1')

In [95]:
1

1

# 5. Merging LinkedIn and O*NET 

In [96]:
title_crosswalk = pd.read_excel(main_link + "Input/Linked_titles_LinkedIn_ONET.xlsx" )

# LinkedIn ONET, Occupations titles
Dict_LinkedIn_ONET_titles = dict(zip(title_crosswalk.LinkedIn_titles, title_crosswalk.ONET_titles))
print(Dict_LinkedIn_ONET_titles["Data Input"])

Data Input Clerk


In [97]:
def crosswalk(x , Dict_LinkedIn_ONET_titles):
  if x in Dict_LinkedIn_ONET_titles.keys():
    output  = Dict_LinkedIn_ONET_titles[x]
  else:
    output = "NA"
  return output 

LinkedIn_2skills_df["ONET_Title"] =  LinkedIn_2skills_df["Title"].apply(lambda x: crosswalk(x , Dict_LinkedIn_ONET_titles))

In [98]:
linkedIn_title_list = list(set(LinkedIn_2skills_df[LinkedIn_2skills_df["ONET_Title"] != "NA"]["ONET_Title"].values.tolist()))

In [99]:
ONET_alternate_short_title = pd.read_excel(main_link + "Input/O*NET Alternate Titles.xlsx")

In [100]:
def change_alternate_short_title(x, d_alternate, d_short):
  if x in d_alternate.keys(): 
    return d_alternate[x]
  elif x in d_short.keys():
    return d_short[x]
  else: return x

Dict_alternate_title = dict(zip(ONET_alternate_short_title["Alternate Title"], ONET_alternate_short_title["Title"] ))
Dict_short_title = dict(zip(ONET_alternate_short_title["Short Title"], ONET_alternate_short_title["Title"] ))
LinkedIn_2skills_df["ONET_Title"] = LinkedIn_2skills_df["ONET_Title"].apply(lambda x: change_alternate_short_title(x,Dict_alternate_title, Dict_short_title))


In [101]:
#Items for which titles could be matched but the element is not in final_ONET
in_LinkedIn_2skills = (LinkedIn_2skills_df[LinkedIn_2skills_df["ONET_Title"] != "NA"]["ONET_Title"].values.tolist())
in_finalONET = list(set(ONET_result["Title"].values.tolist()))
unmatched = set([x for x in in_LinkedIn_2skills if x not in in_finalONET])


In [102]:
related_occpuation = pd.read_excel(main_link + 'Input/O*NET Input/Data_xlsx/Related Occupations.xlsx')

Dict_for_unmatched = {}
for i in range(1,21):
  
  related_occpuation_sub = related_occpuation[related_occpuation["Index"] == i ]
  Dict_current_i = dict(zip(related_occpuation_sub["Related Title"], related_occpuation_sub["Title"] ))
  for j in unmatched:
    if j in Dict_current_i.keys() and j not in Dict_for_unmatched.keys():
      Dict_for_unmatched[j] = Dict_current_i[j]


def change_unmatched(x,d):
  if x in d.keys(): return d[x]
  else: return x

LinkedIn_2skills_df["ONET_Title"] = LinkedIn_2skills_df["ONET_Title"].apply(lambda x: change_unmatched(x,Dict_for_unmatched))


In [103]:
a = 0 
for i in unmatched :
  if i not in Dict_for_unmatched.keys(): 
    # print(i)
    a = a +1

print(a)

50


In [104]:
LinkedIn_2skills_df = LinkedIn_2skills_df.rename(columns={"PC1": "LinkedIn_PC1", "PC2": "LinkedIn_PC2"})

merged = LinkedIn_2skills_df.merge(ONET_result, left_on='ONET_Title', right_on='Title')

In [105]:
merged = merged[["LinkedIn_PC1", "LinkedIn_PC2", "ONET_Title", 	"Title_y", 	"ONET_Cognitive", 	"ONET_Mechanical", 	"ONET_Interpersonal", "Tenure"]]
merged = merged.rename(columns={"PC1": "ONET_Cognitive", "PC2": "ONET_Mechanical","PC3":"ONET_Interpersonal"})


In [106]:
merged

,LinkedIn_PC1,LinkedIn_PC2,ONET_Title,Title_y,ONET_Cognitive,ONET_Mechanical,ONET_Interpersonal,Tenure
0,-0.040760,-0.041465,Machine Feeders and Offbearers,Machine Feeders and Offbearers,-0.188932,0.144213,-0.168046,"[Sep 2015 – Present, 6 yrs 11 mos]"
1,-0.077444,-0.053133,Web Administrators,Web Administrators,0.149224,-0.190304,0.061710,"[Jul 2020 – Feb 2021, 7 mos]"
2,0.002176,0.012200,Web Administrators,Web Administrators,0.149224,-0.190304,0.061710,"[Apr 2012 – May 2013, 1 yr 1 mo]"
3,-0.077444,-0.053133,Web Administrators,Web Administrators,0.149224,-0.190304,0.061710,"[May 1998 – Sep 2001, 3 yrs 4 mos]"
4,-0.077444,-0.053133,Web Administrators,Web Administrators,0.149224,-0.190304,0.061710,"[Jan 2018 – Jun 2018, 5 mos]"
...,...,...,...,...,...,...,...,...
9408,-0.184353,-0.131504,Gambling Dealers,Gambling Dealers,-0.233459,-0.275506,-0.053417,"[2009 – 2014, 5 yrs]"
9409,-0.056579,-0.027226,"Low Vision Therapists, Orientation and Mobilit...","Low Vision Therapists, Orientation and Mobilit...",-0.033989,-0.150673,0.058436,"[Mar 2013 – Oct 2016, 3 yrs 7 mos]"
9410,-0.184353,-0.131504,"Control and Valve Installers and Repairers, Ex...","Control and Valve Installers and Repairers, Ex...",-0.007926,0.418221,-0.094442,"[Oct 2018 – Mar 2019, 5 mos]"
9411,0.029961,0.027402,Natural Sciences Managers,Natural Sciences Managers,0.203642,-0.045470,0.100187,"[May 2019 – Aug 2021, 2 yrs 3 mos]"


In [107]:
with pd.ExcelWriter(main_link + 'Temp/merged.xlsx') as writer:  
    merged.to_excel(writer, sheet_name='Sheet_name_1')


# 6. Regression prep

In [ ]:
def tomonths(x):
  output = None
  if len(x) > 1:
    a = x[1] #.split(",")[0][1:]
    
    split_t = a.strip().split(" ") 
    if len(split_t) == 4 : output = int(split_t[0]) * 12 + int(split_t[2])
    elif len(split_t) == 2 and (split_t[1] == "yr" or split_t[1] == "yrs")  : output = int(split_t[0]) *12
    elif len(split_t) == 2 and (split_t[1] == "mo" or split_t[1] == "mos"  ): output = int(split_t[0])
  return output 

merged["months"] = merged["Tenure"].apply(lambda x: tomonths(x))
merged["Interpersonal_mismatch"] = merged["LinkedIn_PC1"] - merged["ONET_Interpersonal"]

print(len(merged.values))


# with pd.ExcelWriter(main_link + 'Temp/merged.xlsx') as writer:  
#     merged.to_excel(writer, sheet_name='Sheet_name_1')

8729


In [ ]:
model = LinearRegression()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


ValueError: ignored